In [ ]:
import IPython.display
import ipyvolume.pylab as p3d
import matplotlib.cm
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
import time
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import tiledb

In [ ]:
tiledb.libtiledb.version()

In [ ]:
ctx = tiledb.Ctx()

In [ ]:
arr = tiledb.SparseArray.load(ctx, "data/nyu_dublin_lidar")

In [ ]:
arr.domain

In [ ]:
data = arr[:]
data["coords"][:10]

In [ ]:
X = data["coords"]['x'][::10].astype(float)
Y = data["coords"]['y'][::10].astype(float)
Z = data["coords"]['z'][::10].astype(float) * 2 # need to give some height to this state
Xadj = X - X.min()
Yadj = Y - Y.min()
Zadj = Z - Z.min()

In [ ]:
fig = p3d.figure(width=1000)
p3d.style.use(["light", "minimal"])
all_points = p3d.scatter(Yadj, Zadj, Xadj, color='red', size=0.08)
p3d.squarelim()
p3d.show()

In [ ]:
# Select out the interesting columns
df = pd.DataFrame(data=data, columns=["coords", "intensity", "classification", "point_source_id"])
df.head(5)

In [ ]:
# plot the classification numbers with different colors
coords1 = data["coords"][(data["classification"] & 0xf) == 2]
coords2 = data["coords"][(data["classification"] & 0xf) == 4]
print("N points, Class 1: {}".format(len(coords1)))
print("N points, Class 2: {}".format(len(coords2)))

In [ ]:
# Plot the classification points
X1 = coords1['x'][::10].astype(float)
Y1 = coords1['y'][::10].astype(float)
Z1 = coords1['z'][::10].astype(float)
min_ground = Z1.min()
X1 = X1 - X1.min()
Y1 = Y1 - Y1.min()
Z1 = Z1 - min_ground
groundarr = np.column_stack((X1,Y1))
groundtri = Delaunay(groundarr)

X2 = coords2['x'][::10].astype(float)
Y2 = coords2['y'][::10].astype(float)
Z2 = coords2['z'][::10].astype(float)
X2 = X2 - X2.min()
Y2 = Y2 - Y2.min()
Z2 = Z2 - min_ground
vegarr = np.column_stack((X2,Y2))
vegtri = Delaunay(vegarr)

class_fig = p3d.figure(width=1000)
p3d.style.use(["light", "minimal"])
p3d.plot_trisurf(Y1, Z1, X1, triangles=groundtri.simplices, color='grey')
c2_points = p3d.scatter(Y2, Z2, X2, color='green', size=0.03)
class_fig.scatters.append(c2_points)
p3d.squarelim()
p3d.show()

In [ ]:
# Testing slicing and filtering performance
# Query 1
start_time = time.time()
data = arr[:]
print("Query 1, N points: {}".format(len(data["coords"])))
print("Query 1, slicing time: {}".format(time.time()-start_time))
start_time = time.time()
coords1 = data["coords"][(data["classification"] & 0xf) == 2]
print("Query 1, filtering time: {}".format(time.time()-start_time))

In [ ]:
# Query 2
start_time = time.time()
data = arr[2145880000:2146500000, :, :]
print("Query 2, N points: {}".format(len(data["coords"])))
print("Query 2, slicing time: {}".format(time.time()-start_time))
start_time = time.time()
coords1 = data["coords"][(data["classification"] & 0xf) == 2]
print("Query 2, filtering time: {}".format(time.time()-start_time))

In [ ]:
# Query 3
start_time = time.time()
data = arr[2145880000:2146300000, :, :]
print("Query 3, N points: {}".format(len(data["coords"])))
print("Query 3, slicing time: {}".format(time.time()-start_time))
start_time = time.time()
coords1 = data["coords"][(data["classification"] & 0xf) == 2]
print("Query 3, filtering time: {}".format(time.time()-start_time))

In [ ]:
# Query 4
start_time = time.time()
data = arr[2145880000:2146000000, :, :]
print("Query 4, N points: {}".format(len(data["coords"])))
print("Query 4, slicing time: {}".format(time.time()-start_time))
start_time = time.time()
coords1 = data["coords"][(data["classification"] & 0xf) == 2]
print("Query 4, filtering time: {}".format(time.time()-start_time))